#### Change the promting according to the promting of pip-sql-1.3b
https://huggingface.co/PipableAI/pip-sql-1.3b

#### Go from using langchain sql chain to manual promt template.

In [1]:
from sqlalchemy import create_engine, text
from langchain.sql_database import SQLDatabase

db_path = '/home/arseniy/Документы/Projects/Python/LLM_project/json-to-sql/mys_db'

engine = create_engine ('sqlite:///' + db_path, echo= False, future=True)

db = SQLDatabase(engine)


In [2]:
from langfuse.callback import CallbackHandler
langfuse_handler = CallbackHandler(
    public_key="key",
    secret_key="key",
    host="https://cloud.langfuse.com"
)

In [3]:
from langchain_community.chat_models import ChatLlamaCpp 
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
import multiprocessing
from langchain.globals import set_llm_cache
# Path to the model weights
models_directory = '/home/arseniy/Документы/Projects/Python/LLM_project/models/'

local_model = models_directory + 'sqlcoder-7b-2.Q4_K_M.gguf' 


llm = ChatLlamaCpp(
    cache = True,
    model_path = local_model,
    n_ctx=2000,
    n_gpu_layers=10,
    use_mlock = True,
    versbose = True,
    

)


llama_model_loader: loaded meta data with 28 key-value pairs and 291 tensors from /home/arseniy/Документы/Projects/Python/LLM_project/models/sqlcoder-7b-2.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = models
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 16384
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.attention.head_count u32              = 32
llama_model_loader: - kv   7

In [4]:
from operator import itemgetter
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain.chains import LLMChain

In [5]:
with open('mys_schema_copy.txt', 'r') as file:
    description = file.read()


In [6]:
schema = description
question = ""

# Define the prompt template using f-string

template =  """
                Generate a SQL query to answer [QUESTION]{question}[/QUESTION]
                
                The query will run on a database with the following schema:
                {schema}
                
                Given the database schema, here is the SQL query that [QUESTION]{question}[/QUESTION]
                [SQL]
            """
# Initialize the prompt template with placeholders for 'schema' and 'question'
prompt_template = PromptTemplate.from_template(template)


### https://python.langchain.com/v0.2/docs/integrations/llm_caching/

In [7]:
chain = LLMChain(prompt=prompt_template, llm=llm)

/home/arseniy/.var/app/org.jupyter.JupyterLab/config/jupyterlab-desktop/jlab_server/envs/openai/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [8]:
from langchain.cache import InMemoryCache

set_llm_cache(InMemoryCache())

In [ ]:
import sqlvalidator
import sqlcorrect as sc

def main():
    print("🔴 Simple database chatbot app. Type 'exit' to quit.")
    while True:
        # Get user input
        question = input("🟢 Enter your question: ")
        if question.lower() == 'exit':
            print("Exiting the app.")
            break 

        response = chain.run(question = question, schema=description)
        try:            
            print("Response:", response) 
        except:
            print("🔴 LLM agent gave a wrong SQL query. Paraphrase the question...")
        

if __name__ == "__main__":
    main()

🔴 Simple database chatbot app. Type 'exit' to quit.


🟢 Enter your question:  show me names of employees with id 34,22 and 34


Llama.generate: prefix-match hit

llama_print_timings:        load time =     550.61 ms
llama_print_timings:      sample time =      10.78 ms /    26 runs   (    0.41 ms per token,  2411.65 tokens per second)
llama_print_timings: prompt eval time =   56644.40 ms /   710 tokens (   79.78 ms per token,    12.53 tokens per second)
llama_print_timings:        eval time =    4125.48 ms /    25 runs   (  165.02 ms per token,     6.06 tokens per second)
llama_print_timings:       total time =   60803.95 ms /   735 tokens


Response:   SELECT e.name FROM employees e WHERE e.id IN (34, 22, 34);


#### Classify related table names with smaller model -> query the database with related names -> pass table schemas to the sqlcoder.
#### Think about using cache, especially redis db as cache.